In [1]:
import os
os.chdir("../")

In [2]:
from src.hf_dataset.dataset import get_banking_77
from src.hf_models.get_model import  get_distilgpt2, get_distilbert
from src.config.configuration import ConfigurationManager


/home/tess/miniconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
print(torch.__version__)


2.1.2+cu121


In [4]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
)

In [5]:
ds = get_banking_77()

model, tokenizer, config = get_distilbert(task="SequenceClassification",quantization_config=bnb_config,
                                          num_labels=77)

/home/tess/work/learning/transformers/models


ValueError: DistilBertForSequenceClassification does not support `device_map='auto'`. To implement support, the model class needs to implement the `_no_split_modules` attribute.

In [ ]:
!python -m bitsandbytes


In [ ]:
config_manager = ConfigurationManager()
params = config_manager.get_distilbert_config()


In [ ]:
def foo(**kwargs):
    print(kwargs.items())  
    
foo(hello="goldy", world="hello")

In [5]:

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
ds = ds.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
ds = ds.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

ds["train"] = ds["train"].remove_columns(["text"])
ds["test"] = ds["test"].remove_columns(["text"])

In [6]:
from peft import LoraConfig


lora_config = LoraConfig(
    r = 7, # low rank means smaller update matrices with fewer trainable parameters
    target_modules = ["q_lin", "k_lin"],
    bias="none",
    lora_alpha=32,
    
)


In [ ]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [ ]:
model

In [ ]:
model.print_trainable_parameters()

In [16]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import EvalPrediction
import numpy as np

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    torch_compile=True
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    compute_metrics=compute_accuracy,
)

In [ ]:
ds

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(ds["test"])

In [ ]:
model

In [ ]:
import transformers
import datasets
import accelerate
import torch
import peft
import bitsandbytes
import triton

print(f"transformers: {transformers.__version__}")
print(f"datasets: {datasets.__version__}")
print(f"accelerate: {accelerate.__version__}")
print(f"torch: {torch.__version__}")
print(f"peft: {peft.__version__}")
# print(f"bitsandbytes: {bitsandbytes.__version__p}")
print(f"triton: {triton.__version__}")


transformers: 4.35.2
datasets: 2.15.0
accelerate: 0.25.0
torch: 2.1.2+cu121
peft: 0.7.1


AttributeError: module 'bitsandbytes' has no attribute '__version__p'

In [ ]:
torch.cuda.is_available()

In [ ]:
pip install \
  transformers==4.35.2 \
  peft==0.7.1 \
  accelerate==0.25.0 \
  datasets==2.15.0 \
  langchain==0.0.354 \  
  bitsandbytes==0.41.3.post2 \
  triton==2.1.0 \
  crewai==0.1.16
  
  pip install \
  transformers==4.35.2 \
  peft==0.7.1 \
  accelerate==0.25.0 \
  datasets==2.15.0 \
  langchain==0.0.354 \
  bitsandbytes==0.41.3.post2 \
  triton==2.1.0 \
  crewai==0.1.16